<a href="https://colab.research.google.com/github/Wanita-8943/efficientnet_keras_transfer_learning/blob/master/All.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [162]:
import pandas as pd
import shutil

In [166]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [167]:
df = pd.read_csv (r'/content/drive/My Drive/All_Film6.csv')
df

FileNotFoundError: ignored

In [165]:
classes = ['Y7','Y8','Y9','Y10','Y11','Y12','Y13','Y16','Y17','Y18']
len(classes)

10

In [144]:
df.shape

(1841, 7)

In [145]:
print(set(df['Class']))
print(set(df['Sex']))
print(set(df['Floder']))

{'Y10', 'Y17', 'Y7', 'Y13', 'Y11', 'Y8', 'Y16', 'Y9', 'Y12', 'Y18'}
{'เพศหญิง', 'เพศชาย'}
{'Lt', 'Both', 'Rt'}


In [146]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import os
import glob
import shutil
import sys
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline

In [147]:
#กำหนดค่าพารามิเตอร์
batch_size = 15

width = 150
height = 150
epochs = 20
NUM_TRAIN = 1800
NUM_TEST = 200
dropout_rate = 0.2
input_shape = (height, width, 3)

#Clone efficientnet repo


In [148]:
#ดึงข้อมูลใน Github มาใช้
import os
%cd /content
if not os.path.isdir("efficientnet_keras_transfer_learning"):
 !git clone https://github.com/Wanita-8943/efficientnet_keras_transfer_learning
%cd efficientnet_keras_transfer_learning/


/content
/content/efficientnet_keras_transfer_learning


In [149]:
# Options: EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3
# Higher the number, the more complex the model is.
from efficientnet import EfficientNetB0 as Net
from efficientnet import center_crop_and_resize, preprocess_input

In [150]:
# loading pretrained conv base model
# โหลดโมเดล มาโดยตัด output ของโมเดลออก เเต่ยังใช้ input อันเดิม
# เเละโหลด weight ของโมเดล มาด้วยที่ชื่อว่า imagenet
conv_base = Net(weights='imagenet', include_top=False, input_shape=input_shape)

In [151]:
base_dir = '/content/drive/My Drive/Panoramic_Film'
os.makedirs(base_dir, exist_ok=True)

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
os.makedirs(train_dir, exist_ok=True)
validation_dir = os.path.join(base_dir, 'validation')
os.makedirs(validation_dir, exist_ok=True)
test_dir = os.path.join(base_dir, 'test')
os.makedirs(test_dir, exist_ok=True)

In [152]:
train_Y7_dir = os.path.join(train_dir, 'Y7')
os.makedirs(train_Y7_dir, exist_ok=True)

train_Y8_dir = os.path.join(train_dir, 'Y8')
os.makedirs(train_Y8_dir, exist_ok=True)

train_Y9_dir = os.path.join(train_dir, 'Y9')
os.makedirs(train_Y9_dir, exist_ok=True)

train_Y10_dir = os.path.join(train_dir, 'Y10')
os.makedirs(train_Y10_dir, exist_ok=True)

train_Y11_dir = os.path.join(train_dir, 'Y11')
os.makedirs(train_Y11_dir, exist_ok=True)

train_Y12_dir = os.path.join(train_dir, 'Y12')
os.makedirs(train_Y12_dir, exist_ok=True)

train_Y13_dir = os.path.join(train_dir, 'Y13')
os.makedirs(train_Y13_dir, exist_ok=True)

train_Y16_dir = os.path.join(train_dir, 'Y16')
os.makedirs(train_Y16_dir, exist_ok=True)

train_Y17_dir = os.path.join(train_dir, 'Y17')
os.makedirs(train_Y17_dir, exist_ok=True)

train_Y18_dir = os.path.join(train_dir, 'Y18')
os.makedirs(train_Y18_dir, exist_ok=True)

validation_Y7_dir = os.path.join(validation_dir, 'Y7')
os.makedirs(validation_Y7_dir, exist_ok=True)

validation_Y8_dir = os.path.join(validation_dir, 'Y8')
os.makedirs(validation_Y8_dir, exist_ok=True)

validation_Y9_dir = os.path.join(validation_dir, 'Y9')
os.makedirs(validation_Y9_dir, exist_ok=True)

validation_Y10_dir = os.path.join(validation_dir, 'Y10')
os.makedirs(validation_Y10_dir, exist_ok=True)

validation_Y11_dir = os.path.join(validation_dir, 'Y11')
os.makedirs(validation_Y11_dir, exist_ok=True)

validation_Y12_dir = os.path.join(validation_dir, 'Y12')
os.makedirs(validation_Y12_dir, exist_ok=True)

validation_Y13_dir = os.path.join(validation_dir, 'Y13')
os.makedirs(validation_Y13_dir, exist_ok=True)

validation_Y16_dir = os.path.join(validation_dir, 'Y16')
os.makedirs(validation_Y16_dir, exist_ok=True)

validation_Y17_dir = os.path.join(validation_dir, 'Y17')
os.makedirs(validation_Y17_dir, exist_ok=True)

validation_Y18_dir = os.path.join(validation_dir, 'Y18')
os.makedirs(validation_Y18_dir, exist_ok=True)



In [153]:
df

,Fig,Age(year),Class,Filename,Path_filename,Sex,Floder
0,1,7,Y7,V1.jpg,/content/drive/My Drive/Film pano/7 year/7Y-F/...,เพศหญิง,Both
1,2,7,Y7,V2.jpg,/content/drive/My Drive/Film pano/7 year/7Y-F/...,เพศหญิง,Both
2,3,7,Y7,V3.jpg,/content/drive/My Drive/Film pano/7 year/7Y-F/...,เพศหญิง,Both
3,4,7,Y7,V6.jpg,/content/drive/My Drive/Film pano/7 year/7Y-F/...,เพศหญิง,Both
4,5,7,Y7,V7.jpg,/content/drive/My Drive/Film pano/7 year/7Y-F/...,เพศหญิง,Both
...,...,...,...,...,...,...,...
1836,143,18,Y18,S264.jpg,/content/drive/My Drive/Film pano/18 year/18Y ...,เพศชาย,Both
1837,144,18,Y18,S268.jpg,/content/drive/My Drive/Film pano/18 year/18Y ...,เพศชาย,Both
1838,145,18,Y18,S270.jpg,/content/drive/My Drive/Film pano/18 year/18Y ...,เพศชาย,Both
1839,146,18,Y18,S274.jpg,/content/drive/My Drive/Film pano/18 year/18Y ...,เพศชาย,Both


In [154]:
val = df[df['Fig'].between(161,180)]
train = df[df['Fig'].between(1,160)]

Y7_train = train[train['Class']=='Y7' ]
Y7_path_train = Y7_train['Path_filename'].tolist() 
Y8_train = train[train['Class']=='Y8' ]
Y8_path_train = Y8_train['Path_filename'].tolist() 
Y9_train = train[train['Class']=='Y9' ]
Y9_path_train = Y9_train['Path_filename'].tolist()
Y10_train = train[train['Class']=='Y10' ]
Y10_path_train = Y10_train['Path_filename'].tolist()
Y11_train = train[train['Class']=='Y11' ]
Y11_path_train = Y11_train['Path_filename'].tolist()
Y12_train = train[train['Class']=='Y12' ]
Y12_path_train = Y12_train['Path_filename'].tolist()
Y13_train = train[train['Class']=='Y13' ]
Y13_path_train = Y13_train['Path_filename'].tolist()
Y16_train = train[train['Class']=='Y16' ]
Y16_path_train = Y16_train['Path_filename'].tolist() 
Y17_train = train[train['Class']=='Y17' ]
Y17_path_train = Y17_train['Path_filename'].tolist()
Y18_train = train[train['Class']=='Y18' ]
Y18_path_train = Y18_train['Path_filename'].tolist()

Y7_val = val[val['Class']=='Y7' ]
Y7_path_val = Y7_val['Path_filename'].tolist() 
Y8_val = val[val['Class']=='Y8' ]
Y8_path_val = Y8_val['Path_filename'].tolist() 
Y9_val = val[val['Class']=='Y9' ]
Y9_path_val = Y9_val['Path_filename'].tolist()
Y10_val = val[val['Class']=='Y10' ]
Y10_path_val = Y10_val['Path_filename'].tolist()
Y11_val = val[val['Class']=='Y11' ]
Y11_path_val = Y11_val['Path_filename'].tolist()
Y12_val = val[val['Class']=='Y12' ]
Y12_path_val = Y12_val['Path_filename'].tolist()
Y13_val = val[val['Class']=='Y13' ]
Y13_path_val = Y13_val['Path_filename'].tolist()
Y16_val = val[val['Class']=='Y16' ]
Y16_path_val = Y16_val['Path_filename'].tolist() 
Y17_val = val[val['Class']=='Y17' ]
Y17_path_val = Y17_val['Path_filename'].tolist()
Y18_val = val[val['Class']=='Y18' ]
Y18_path_val = Y18_val['Path_filename'].tolist()

In [155]:
fnames = Y7_path_train
for fname in fnames:
    dst = os.path.join(train_Y7_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

fnames = Y8_path_train
for fname in fnames:
    dst = os.path.join(train_Y8_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)
    
fnames = Y9_path_train  
for fname in fnames:
    dst = os.path.join(train_Y9_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

fnames = Y10_path_train 
for fname in fnames:
    dst = os.path.join(train_Y10_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)
    
fnames = Y11_path_train
for fname in fnames:
    dst = os.path.join(train_Y11_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)  

fnames = Y12_path_train  
for fname in fnames:
    dst = os.path.join(train_Y12_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)  
    
fnames = Y13_path_train
for fname in fnames:
    dst = os.path.join(train_Y13_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

    fnames = Y16_path_train  
for fname in fnames:
    dst = os.path.join(train_Y16_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)  

fnames = Y17_path_train
for fname in fnames:
    dst = os.path.join(train_Y17_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)  
    
fnames = Y18_path_train
for fname in fnames:
    dst = os.path.join(train_Y18_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

FileNotFoundError: ignored

In [ ]:
fnames = Y7_path_val
for fname in fnames:
    dst = os.path.join(validation_Y7_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

fnames = Y8_path_val  
for fname in fnames:
    dst = os.path.join(validation_Y8_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)
    
fnames = Y9_path_val  
for fname in fnames:
    dst = os.path.join(validation_Y9_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

fnames = Y10_path_val  
for fname in fnames:
    dst = os.path.join(validation_Y10_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)
    
fnames = Y11_path_val  
for fname in fnames:
    dst = os.path.join(validation_Y11_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)  

fnames = Y12_path_val  
for fname in fnames:
    dst = os.path.join(validation_Y12_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)  
    
fnames = Y13_path_val  
for fname in fnames:
    dst = os.path.join(validation_Y13_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

    fnames = Y16_path_val  
for fname in fnames:
    dst = os.path.join(validation_Y16_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)  

fnames = Y17_path_val
for fname in fnames:
    dst = os.path.join(validation_Y17_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)  
    
fnames = Y18_path_val  
for fname in fnames:
    dst = os.path.join(validation_Y18_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

In [132]:
print('total training 7years images:', len(os.listdir(train_Y7_dir))) 
#print('total training 8years images:', len(os.listdir(train_9years_F_dir)))
#print('total training 9years images:', len(os.listdir(train_10years_F_dir)))
#print('total validation 7years images:', len(os.listdir(validation_8years_F_dir)))
#print('total validation 8years images:', len(os.listdir(validation_9years_F_dir)))
#print('total validation 9years images:', len(os.listdir(validation_10years_F_dir)))

#print('total test 8years_F images:', len(os.listdir(test_8years_F_dir)))
#print('total test 9years_F images:', len(os.listdir(test_9years_F_dir)))
#print('total test 10years_F images:', len(os.listdir(test_10years_F_dir)))


total training 7years images: 0
